In [1]:
# load config
from pathlib import Path
import mlflow
from src.config import parse_config
import os

root_dir = Path(os.getcwd()).parent
config_path = root_dir / 'agents' / 'langgraph' / 'config.yaml'
agent_path = root_dir / 'agents' / 'langgraph' / 'agent.py'

mlflow_config = mlflow.models.ModelConfig(development_config=config_path)
sls_config = parse_config(mlflow_config)

In [2]:
from src.retrievers import get_vector_retriever
retriever = get_vector_retriever(sls_config)

from databricks_langchain import ChatDatabricks
model = ChatDatabricks(endpoint=sls_config.model.endpoint_name)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Let's setup some nodes to play with

In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from src.states import get_state
from src.nodes import (
    make_query_vector_database_node, 
    make_context_generation_node,
    make_simple_generation_node
)

state = get_state(sls_config)
retriever_node = make_query_vector_database_node(retriever, sls_config)
simple_generation_node = make_simple_generation_node(model, sls_config)
context_generation_node = make_context_generation_node(model, sls_config)

This section builds a simple generation graph. It expects an input state with a dictionary of messages.

In [10]:
input_example = {'messages':[{'type':'user', 'content':'I want to ship milk using a train, but goats milk not cow milk'}]}

We use langchain's convert_to_messages to convert the input state to a list of messages. This is convenient because OpenAI uses 'role' and LangChain uses 'type'. We centralize on LangChain's message type for now.

This expects a list of dictionaries with 'type' and 'content' keys. Will fail is the entire {'messages':[{'type':'user', 'content':'What is SQL?'}]} is passed in.

We can use the convert_to_openai_messages function to convert the list of LangChain messages back to a list of dictionaries with 'role' and 'content' keys.

```python
from langchain_core.messages.utils import convert_to_messages
lc_msgs = convert_to_messages(input_example['messages'])
```

In [11]:
workflow = StateGraph(state)
workflow.add_node("generate", simple_generation_node)
workflow.add_edge(START, "generate")
workflow.add_edge("generate", END)
app = workflow.compile()
app.invoke(input_example)

{'messages': [{'type': 'user',
   'content': 'I want to ship milk using a train, but goats milk not cow milk'},
  {'role': 'assistant',
   'content': "So, yeah. That's an interesting choice. Shipping goat's milk by train can be a bit more challenging than shipping cow's milk, as goat's milk is often produced in smaller quantities and may require more specialized handling and storage. However, it's definitely doable. You'll want to make sure you're working with a refrigerated train car to keep the milk at a safe temperature during transport, and you may need to consider using specialized packaging or containers to prevent the milk from spoiling or being damaged during the journey. Additionally, you'll need to research any regulations or certifications required for shipping goat's milk by train, such as those related to food safety and handling. It's also important to consider the logistics of loading and unloading the milk at the train stations, as well as the overall cost and efficienc

Check context generation node with no context

In [12]:
workflow = StateGraph(state)
workflow.add_node("generate", context_generation_node)
workflow.add_edge(START, "generate")
workflow.add_edge("generate", END)
app = workflow.compile()
app.invoke(input_example)

{'messages': [{'type': 'user',
   'content': 'I want to ship milk using a train, but goats milk not cow milk'},
  {'role': 'assistant',
   'content': "That's an interesting request. Unfortunately, I don't have any specific information about shipping goat's milk by train, and the provided context is empty, so I don't know the details of shipping goat's milk by train. The context is: there is no context provided."}]}

Check retriever only


In [13]:
workflow = StateGraph(state)
workflow.add_node("retrieve", retriever_node)
workflow.add_edge(START, "retrieve")
app = workflow.compile()
app.invoke(input_example)

{'messages': [{'type': 'user',
   'content': 'I want to ship milk using a train, but goats milk not cow milk'}],
 'context': 'Passage: 3 “plant” means a cream transfer station, a milk transfer station or premises in which milk or cream or milk products are \nprocessed; (“usine”)  \n“processing” means heating, pasteurizing, evaporating, drying, churning, freezing, packaging, packing, separating into \ncomponent parts, combining with other substances by any process or otherwise treating milk or cream or milk products in \nthe manufacture or preparation of milk products or fluid milk products; (“transformation”)  \n“processor” means a person engaged in the processing of milk products or fluid milk products; (“préposé à la \ntransformation”)  \n“producer” means a producer of milk, cream or cheese; (“producteur”)  \n“reconstituted milk” means milk designated as reconstituted milk in the regulations; (“lait reconstitué”)  \n“regulated product” means milk, cream or cheese, or any combination 

This section builds a rag graph without history

In [14]:
workflow = StateGraph(state)
workflow.add_node("retrieve", retriever_node)
workflow.add_node("generate_w_context", context_generation_node)
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "generate_w_context")
workflow.add_edge("generate_w_context", END)
app = workflow.compile()
result = app.invoke(input_example)

In [15]:
from src.utils import graph_state_to_chat_type
from langchain_core.runnables import RunnableLambda

chain = app | RunnableLambda(graph_state_to_chat_type)
chain.invoke(input_example)

{'choices': [{'message': {'role': 'assistant',
    'content': "The provided context appears to be related to Ontario's milk regulation and transportation laws, but it does not specifically address shipping goat's milk by train.",
    'refusal': None,
    'name': None,
    'tool_calls': None,
    'tool_call_id': None},
   'index': 0,
   'finish_reason': 'stop',
   'logprobs': None}],
 'usage': None,
 'id': None,
 'model': None,
 'object': 'chat.completion',
 'created': 1739568994,
 'custom_outputs': {'message_history': [{'type': 'user',
    'content': 'I want to ship milk using a train, but goats milk not cow milk'},
   {'role': 'tool',
    'content': 'Passage: 3 “plant” means a cream transfer station, a milk transfer station or premises in which milk or cream or milk products are \nprocessed; (“usine”)  \n“processing” means heating, pasteurizing, evaporating, drying, churning, freezing, packaging, packing, separating into \ncomponent parts, combining with other substances by any proces